In [1]:
import re
import os
import sys
import time
import openai
import warnings
import pandas as pd
from openai import OpenAI

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)
sys.path.append('/home/or/dev/latin_music_statistics/crawler/data_dir')
sys.path.append(os.path.abspath(os.path.join(parent_dir, 'statistics_dir')))
sys.path.append(os.path.abspath(os.path.join(parent_dir, 'process_data_dir')))

from config import *
from helper import *

client = OpenAI(api_key = OPENAI_KEY)

In [7]:
df=pd.read_csv('final_data.csv')

In [17]:
for idx,row in df[['title_with_artists','theme']].iterrows():
    print('title_with_artists: ', row['title_with_artists'])
    print('theme: ', row['theme'])
    print(idx,'-'*100)

    if idx ==99:
        break

title_with_artists:  Obsesión by Aventura (Ft. Judy Santos)
theme:  A man's obsession with a woman, mistaking his intense infatuation for love.
0 ----------------------------------------------------------------------------------------------------
title_with_artists:  La Romana by Bad Bunny (Ft. El Alfa)
theme:  A lively and energetic party atmosphere, emphasizing street life, wealth, and enjoying the night without worries.
1 ----------------------------------------------------------------------------------------------------
title_with_artists:  Ayer Me Llamó Mi Ex by KHEA (Ft. Lenny Santos)
theme:  Nostalgia and regret over a past relationship, where physical connection overshadowed emotional issues, leading to longing for the intensity and passion that is now missing with a current partner.
2 ----------------------------------------------------------------------------------------------------
title_with_artists:  Volví by Aventura & Bad Bunny
theme:  A passionate and possessive love wh

In [8]:
countes =0 
for idx,row in df.iterrows():
    sent = eval(row['sentiment'])
    if sent['positive'] == sent['negative']:
        countes +=1
    

In [4]:
df2 = df.copy(deep=True)

In [5]:

def get_dominant_sentiment(sentiment_dict_input):
    try:
        sentiment_dict = eval(sentiment_dict_input)
    except TypeError as e:
        sentiment_dict = sentiment_dict_input

    if sentiment_dict['positive'] == sentiment_dict['negative']:
        print(sentiment_dict_input)
        dominant_sentiment = 'neutral'
    else:
        dominant_sentiment = max(sentiment_dict, key=sentiment_dict.get)  # Get the key with the highest value
    return dominant_sentiment


def calc_sentiment(df):
    df['selected_sentiment'] = df['sentiment'].apply(get_dominant_sentiment)


calc_sentiment(df2)

{'neutral': 0.1, 'positive': 0.45, 'negative': 0.45}
{'neutral': 0.2, 'positive': 0.4, 'negative': 0.4}
{'neutral': 0.3, 'positive': 0.35, 'negative': 0.35}
{'neutral': 0.2, 'positive': 0.4, 'negative': 0.4}
{'neutral': 0.2, 'positive': 0.4, 'negative': 0.4}
{'neutral': 0.2, 'positive': 0.4, 'negative': 0.4}
{'neutral': 0.2, 'positive': 0.4, 'negative': 0.4}
{'neutral': 0.2, 'positive': 0.4, 'negative': 0.4}
{'neutral': 0.2, 'positive': 0.4, 'negative': 0.4}
{'neutral': 0.2, 'positive': 0.4, 'negative': 0.4}
{'neutral': 0.4, 'positive': 0.3, 'negative': 0.3}
{'neutral': 0.3, 'positive': 0.35, 'negative': 0.35}
{'neutral': 0.3, 'positive': 0.35, 'negative': 0.35}


In [6]:
df2.to_csv('final_data.csv')

In [41]:
df3.drop(['general_theme_emb','Unnamed: 0'],axis=1,inplace=True)


In [9]:
df[['title_with_artists', 'sentiment', 'theme', 'general_theme']]

,title_with_artists,sentiment,theme,general_theme
0,Obsesión by Aventura (Ft. Judy Santos),"{'neutral': 0.2, 'positive': 0.1, 'negative': ...","A man's obsession with a woman, mistaking his ...",Toxic Relationships
1,La Romana by Bad Bunny (Ft. El Alfa),"{'neutral': 0.3, 'positive': 0.5, 'negative': ...","A lively and energetic party atmosphere, empha...",Wealth and Materialism
2,Ayer Me Llamó Mi Ex by KHEA (Ft. Lenny Santos),"{'neutral': 0.1, 'positive': 0.45, 'negative':...","Nostalgia and regret over a past relationship,...",Nostalgia and Reflection
3,Volví by Aventura & Bad Bunny,"{'neutral': 0.2, 'positive': 0.5, 'negative': ...",A passionate and possessive love where the sin...,Toxic Relationships
4,LA FAMA by ROSALÍA (Ft. The Weeknd),"{'neutral': 0.2, 'positive': 0.05, 'negative':...","The deceptive and transient nature of fame, an...",Fame and Deception
...,...,...,...,...
838,Xiomara by El Cuarto Bate,"{'neutral': 0.1, 'positive': 0.05, 'negative':...",Unrequited love and the deep emotional pain an...,Heartbreak and Loss
839,La Chancletera by El Cuarto Bate,"{'neutral': 0.1, 'positive': 0.05, 'negative':...",Exposing the deceitful nature of someone who t...,Fame and Deception
840,Mil Vidas by Tony Lozano (Ft. DerekVinci),"{'neutral': 0.1, 'positive': 0.15, 'negative':...",A man's assertion of his irreplaceable role in...,Critique of Love
841,Déjame Tener Mi Novia by Blas Duran JR,"{'neutral': 0.2, 'positive': 0.7, 'negative': ...",The theme of the song is: A man repeatedly ask...,Humor and Light-Heartedness


In [43]:
df3.to_csv('themes/general_themes.csv', index = False)

In [18]:
MAX_THEME_CLUSTER_SIZE = 20
THEME_BATCH_SIZE = 100

SONGS_CLUSTERING_PROMPT = "Cluster the following song themes into brief general themes:\n\n{batch_text}\n\nPlease provide the clusters in a clean Python list format without any extra text or formatting. The list should be written exactly as a Python list, like this: [\"...\",\"...\",\"...\"].\n\nEnsure there are no additional markers or explanations—just the list."
CLUSTERS_PROMPT = f'These song theme clusters\nGroup these song themes into broader, more concise categories, combining similar themes and removing redundancy.\nCombine them into less than {MAX_THEME_CLUSTER_SIZE} ' + 'categories\nThe output should look like: ["...","...","..."].\n{batch_text}\nEnsure all clusters are combined into a single, flat Python list with no nested lists.'

OPENAI_MODEL = "gpt-4o"
import ast

def chatgpt_request_list_extraction(msgs, model_name):
    response = client.chat.completions.create(model=model_name, messages=msgs)
    return response
    
def extract_valid_list(response_text, model_name, max_attempts=5):
    attempt = 0
    chatgpt_messages = []

    while attempt < max_attempts:

        try:
            extracted_list = ast.literal_eval(response_text)
            if isinstance(extracted_list, list):
                return extracted_list
                
        except (SyntaxError, ValueError):
            print(f"Attempt: {attempt + 1} to exrect the python list form the text")
            attempt += 1
            prev_content = f"The previous {attempt + 1} attempt to extract of the valid python list from the text failed.\n Wxtract valid Python list with no additional text, formatting, or code. The output should look exactly like this: [\"...\"].\nProvide only the list, nothing else."
            chatgpt_messages = prepare_chatgpt_msg(response_text, prev_content, chatgpt_messages)
            extraction_response = chatgpt_request_list_extraction(chatgpt_messages, model_name)
            response_text = extraction_response.choices[0].message.content
    
    raise ValueError("Failed to extract a valid Python list after multiple attempts.")

def prepare_chatgpt_msg(curr_text, prev_text, chatgpt_messages):
    chatgpt_messages.append({"role": "user", "content": prev_text})
    chatgpt_messages.append({"role": "user", "content": curr_text})
    return chatgpt_messages

def python_list_to_batch_text_promt(base_prompt, a_list, prev_text, batch_size, row_i, max_len):
    batch_text = "\n".join(a_list[row_i: min(row_i + batch_size, max_len)])
    prompt = base_prompt.format(batch_text=batch_text)
    chatgpt_msg_request =  prepare_chatgpt_msg(prompt, prev_text, [])
    return chatgpt_msg_request

def check_max_clusters(failed_extraction, max_theme_clusters_size ,len_clusters_before):
    return (not failed_extraction) and (max_theme_clusters_size < len_clusters_before)


def batch_process_themes(list_to_cluster, prev_text, base_prompt, theme_batch_size, model):
    row_i = 0
    total_clusters = []
    total_len =len(list_to_cluster)
    
    while row_i <= total_len:
        chatgpt_msg_request = python_list_to_batch_text_promt(base_prompt, list_to_cluster, prev_text, theme_batch_size, row_i, total_len)
        response = chatgpt_request_list_extraction(chatgpt_msg_request, model)
        curr_batch_size = theme_batch_size

        # Check if the response is not finished properly
        while response.choices[0].finish_reason != "stop":
            print(f"Batch {row_i} to {row_i + curr_batch_size} didn't complete. Reducing batch size.")
            curr_batch_size = max(1, curr_batch_size // 2)  # Reduce batch size by half, but not below 1
            chatgpt_msg_request = python_list_to_batch_text_promt(base_prompt, batch_list, prev_text, theme_batch_size, row_i, total_len)
            response = chatgpt_request_list_extraction(chatgpt_msg_request, model)

        response_text = response.choices[0].message.content

        try:
            response_as_list = extract_valid_list(response_text, OPENAI_MODEL, max_attempts=5)
        except ValueError as e:
            print(f"Failed to extract valid list from CHATGPT prompt")
            return(total_clusters) ,1
            
        total_clusters.extend(response_as_list)
        print(f"Processed batch {row_i} to {row_i + THEME_BATCH_SIZE}")

        row_i += curr_batch_size
        
    return total_clusters, 0 

prev_text = ''
song_list = df['theme'].tolist()

clustered_themes, failed_extraction = batch_process_themes(song_list, prev_text, SONGS_CLUSTERING_PROMPT, THEME_BATCH_SIZE, OPENAI_MODEL)

len_clusters_before = len(clustered_themes)
continue_clustring = check_max_clusters(failed_extraction, MAX_THEME_CLUSTER_SIZE, len_clusters_before)


decreasing_clusters_attempt = 1
print(f"Initial number of clusters: {len_clusters_before}")

while continue_clustring:
    curr_clustered_themes, failed_extraction = batch_process_themes(clustered_themes, prev_text, CLUSTERS_PROMPT, THEME_BATCH_SIZE, OPENAI_MODEL)
    prev_clusters_text = '\n'.join(clustered_themes)
    curr_clusters_text = '\n'.join(curr_clustered_themes)
    len_clusters_after = len(curr_clustered_themes)
    
    print(f"The number of clusters: {len_clusters_after}")

    if len_clusters_before == len_clusters_after:
        decreasing_clusters_attempt += 1
        prev_text =  f'This is the {decreasing_clusters_attempt} attempt for clustering. ' +\
                     f'There are {len_clusters_after} many clusters - which are too many. ' +\
                     f'The previous clustered topic attempt was:\n{prev_clusters_text}\n\n\nTry again with those topics:\n' + curr_clusters_text
    else:
        decreasing_clusters_attempt = 1
        prev_text = ''

    continue_clustring = check_max_clusters(failed_extraction, MAX_THEME_CLUSTER_SIZE, len_clusters_after)

    clustered_themes = curr_clustered_themes
    len_clusters_before = len_clusters_after

print(f"The final number of clusters: {len(clustered_themes)}")

Attempt: 1 to exrect the python list form the text
Processed batch 0 to 100
Attempt: 1 to exrect the python list form the text
Processed batch 100 to 200
Processed batch 200 to 300
Processed batch 300 to 400
Processed batch 400 to 500
Attempt: 1 to exrect the python list form the text
Processed batch 500 to 600
Attempt: 1 to exrect the python list form the text
Processed batch 600 to 700
Attempt: 1 to exrect the python list form the text
Processed batch 700 to 800
Processed batch 800 to 900
Initial number of clusters: 585
Processed batch 0 to 100
Processed batch 100 to 200
Attempt: 1 to exrect the python list form the text
Processed batch 200 to 300
Attempt: 1 to exrect the python list form the text
Processed batch 300 to 400
Processed batch 400 to 500
Processed batch 500 to 600
The number of clusters: 247
Processed batch 0 to 100
Attempt: 1 to exrect the python list form the text
Processed batch 100 to 200
Processed batch 200 to 300
The number of clusters: 152
Processed batch 0 to 100

# create specific song themes embeddings and match the closest general theme embeddings

In [5]:
general_themes_df = pd.read_csv('/home/or/dev/latin_music_statistics/crawler/data_dir/themes/general_themes.csv'
)

In [10]:
from sentence_transformers import SentenceTransformer
from tqdm.autonotebook import tqdm
import time

# Initialize the Hugging Face model
MODEL_NAME = 'sentence-transformers/all-roberta-large-v1'

emb_model = SentenceTransformer(ROBERTA_EMBEDDINGS_MODEL)

def create_embeddings(df, model, src_col, tgt_col):
    df[tgt_col] = None
    
    tic = time.time()
    
    # Use tqdm to add a progress bar to the loop
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
        response_emb = model.encode(row[src_col])    
        df.at[index, tgt_col] = response_emb
    
    # Track the end time
    toc = time.time()
    
    # Print the time taken in minutes
    print(f"Time taken: {(toc - tic) / 60:.2f} minutes")

create_embeddings(df, emb_model, 'theme', 'theme_emb')
create_embeddings(general_themes_df,  emb_model, 'general_theme', 'general_theme_emb')

/home/or/dev/latin_music_statistics/lms/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Processing rows:   0%|          | 0/843 [00:00<?, ?it/s]

Time taken: 10.84 minutes


Processing rows:   0%|          | 0/19 [00:00<?, ?it/s]

Time taken: 0.09 minutes


In [12]:
import pandas as pd
from scipy.spatial.distance import cosine
import numpy as np

def find_closest_embeddings(df, general_themes_df):
    # Assuming embeddings are stored as lists or numpy arrays in the DataFrame columns
    df['closest_general_theme'] = None

    for idx, theme_emb in df.iterrows():
        current_embedding = theme_emb['theme_emb']
        
        # Initialize variables to find the closest match
        min_dist = float('inf')
        closest_theme = None
        
        # Iterate over general_themes_df to find the closest match
        for _, general_row in general_themes_df.iterrows():
            general_embedding = general_row['general_theme_emb']
            general_theme = general_row['general_theme']
            
            # Compute cosine similarity (distance) between the current embedding and the general embeddings
            dist = cosine(current_embedding, general_embedding)

            # Update closest match if a smaller distance is found
            if dist < min_dist:
                min_dist = dist
                closest_theme = general_theme
        
        # Store the closest theme in the df
        df.at[idx, 'general_theme'] = closest_theme

    return df

result_df = find_closest_embeddings(df, general_themes_df)

In [23]:
unnamed_columns = [col for col in result_df.columns if 'Unnamed' in col]
result_df.drop(unnamed_columns, axis=1, inplace=True)

In [37]:
# result_df.to_csv('final_data.csv', index=False)

In [48]:
len(df["general_theme"].unique())

19

# Match the general categry to each theme:

In [27]:
from tqdm.notebook import tqdm
tqdm.pandas()

MATCH_PROMPT = 'To which of the following general categories:\n{general_themes}\nThis theme fits best:\n{cur_theme}\nwrite only the catagory name:'
WRONG_GENERAL_THEME =  'This is the {attempt_num} attempt.\n' +\
             'You failed to extract the best fitting general theme out of the theme list.\n' +\
             'You wrote name: "{name}" , which is not in the list.'

df_general_themes = pd.read_csv('/home/or/dev/latin_music_statistics/crawler/data_dir/themes/general_themes.csv')

unnamed_columns = [col for col in df.columns if 'Unnamed' in col]
df.drop(unnamed_columns, axis=1, inplace=True)

def chatgpt_request_list_extraction(msgs, model_name):
    response = client.chat.completions.create(model=model_name, messages=msgs)
    return response
    
def prepare_chatgpt_msg(curr_text, prev_text, chatgpt_messages):
    chatgpt_messages.append({"role": "user", "content": prev_text})
    chatgpt_messages.append({"role": "user", "content": curr_text})
    return chatgpt_messages

def find_generic_theme(theme_txt, general_themes):
    attempt_num = 1
    curr_text = MATCH_PROMPT.format(general_themes=','.join(general_themes), cur_theme=theme_txt)
    chatgpt_msg_request = prepare_chatgpt_msg(curr_text, '', [])
    response = chatgpt_request_list_extraction(chatgpt_msg_request, OPENAI_MODEL)

    response_txt = response.choices[0].message.content
    while response_txt not in general_themes:
        attempt_num += 1
        prev_text = WRONG_GENERAL_THEME.format(attempt_num=attempt_num, name=response_txt)

        chatgpt_msg_request = prepare_chatgpt_msg(curr_text, prev_text, [])
        response = chatgpt_request_list_extraction(chatgpt_msg_request, OPENAI_MODEL)
        response_txt = response.choices[0].message.content

        print(f'{attempt_num}:\n\nprev_text {prev_text}')
        print('-'*100)
    return response_txt  

def match_generic_theme(df, df_general_themes, tgt_theme_col_name):
    df[tgt_theme_col_name] = ""
    general_themes = df_general_themes['general_theme'].tolist()
    
    # Use tqdm to add a progress bar to the loop
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
        result = find_generic_theme(row['theme'], general_themes)
        df.at[index, tgt_theme_col_name] = result

    return df

match_generic_theme(df, df_general_themes, 'general_theme')

Processing rows:   0%|          | 0/843 [00:00<?, ?it/s]

In [32]:
df.columns

Index(['url', 'title_with_artists', 'title', 'artists', 'featured_artists',
       'lyrics', 'clean_lyrics', 'norm_words', 'norm_w_len', 'sentiment',
       'selected_sentiment', 'all_artists', 'theme', 'theme_emb',
       'general_theme', 'closest_general_theme', 'general_theme_openai'],
      dtype='object')

In [42]:
for idx, row in  df[df['selected_sentiment'] == 'positive'][df['general_theme'] == 'Heartbreak and Loss'].iterrows():
    print(row['title'],row['sentiment'])
    print('Theme: ',row['theme'])
    print('Theme: ',row['general_theme'])
    # print('Theme(openai): ',row['general_theme_openai'])
    print('\n')
    print('-' * 100)
    print('\n')

    # if idx >30:
    #     break

EL BARCO {'neutral': 0.15, 'positive': 0.55, 'negative': 0.3}
Theme:  Overcoming heartbreak and moving on from a past relationship.
Theme:  Heartbreak and Loss


----------------------------------------------------------------------------------------------------


Ya Me Olvide De Ti {'positive': 0.65, 'neutral': 0.2, 'negative': 0.15}
Theme:  Overcoming heartbreak and moving on from a past relationship.
Theme:  Heartbreak and Loss


----------------------------------------------------------------------------------------------------


La Bachata {'neutral': 0.3, 'positive': 0.6, 'negative': 0.1}
Theme:  Overcoming heartbreak and embracing newfound freedom and joy in single life through dancing and self-empowerment.
Theme:  Heartbreak and Loss


----------------------------------------------------------------------------------------------------


La Esquinita {'neutral': 0.2, 'positive': 0.8, 'negative': 0.0}
Theme:  Embracing the cyclical nature of life, finding strength and healing in 

/tmp/ipykernel_10122/2531911348.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for idx, row in  df[df['selected_sentiment'] == 'positive'][df['general_theme'] == 'Heartbreak and Loss'].iterrows():


In [11]:
MATCH_PROMPT = 'To which of the following general categories:\n{general_themes}\nThis theme fits best:\n{cur_theme}\nwrite only the catagory name:'
WRONG_GENERAL_THEME =  'This is the {attempt_num} attempt.\n' +\
             'You failed to extract the best fitting general theme out of the theme list.\n' +\
             'You wrote name {name} which is not in the list.' + ' '


In [14]:
print(WRONG_GENERAL_THEME)

This is the {attempt_num} attempt.
You failed to extract the best fitting general theme out of the theme list.
You wrote name {name} which is not in the list. 
